In [51]:
import csv
import tarfile
import io
import psycopg2
import pandas as pd

In [55]:
class Course:
    def __init__(self, cid, term):
        self.cid = cid
        self.term = term
        self.info = {}
        self.meetings = []
        self.enrollment = pd.DataFrame(columns=['SEAT', 'SID', 'SURNAME', 'PREFNAME', 'LEVEL', 'UNITS', 'CLASS', 'MAJOR', 'GRADE', 'STATUS', 'EMAIL'])
    
    def add_meeting(self, meeting):
        if type(meeting) == dict:
            self.meetings.append(meeting)
        elif type(meeting) == list:
            self.meetings.extend(meeting)
            
    def add_enrollment(self, df):
        self.enrollment = df
            
    def add_course_info(self, course_info):
        self.info = course_info

In [56]:
def determine_row_type(big_list):
    if big_list[0][0] == "CID":
        return("CID")
        # class information
        # read one more line to get class information
        
    elif big_list[0][0] == "INSTRUCTOR(S)":
        return("meet")
        
    elif big_list[0][0] == "SEAT":
        return("enroll")
    else:
        print("Don't know how to handle " + str(big_list[0][0]))

In [99]:
def parse_CID(big_list):
    d = {}
    
    if len(big_list) > 2:
            print("List longer than 2 " + str(big_list))
            
    if len(big_list) == 1:
        d = {}
        d["CID"] = None
        d["TERM"] = None
        d["SUBJ"] = None
        d["CRSE"] = None
        d["SEC"] = None
        d["UNITS"] = None
        return(d)
    else:
        try:
            li = big_list[1]
            if len(li) != 6:
                print("Length of Course is not 6. Re-evaluate " + str(li))
            d["CID"] = li[0]
            d["TERM"] = li[1]
            d["SUBJ"] = li[2]
            d["CRSE"] = li[3]
            d["SEC"] = li[4]
            d["UNITS"] = li[5]
            return(d)

        except:
            print("Error when parsing CID")

In [118]:
def parse_meeting(big_list):
    d = {}
    if len(big_list) == 1: # no listings
        d["INSTR"] = None
        d["TYPE"] = None
        d["DAYS"] = None
        d["TIME"] = None
        d["BUILD"] = None
        d["ROOM"] = None
        return d
    
    try:
        meeting_list = []
        for idx, li in enumerate(big_list):
            if idx == 0: # skip over header
                continue
            d = {}
            if idx == 1:
                first_instructor = li[0]
            if li[0] == '' and idx > 0:
                d["INSTR"] = first_instructor
            else:
                d["INSTR"] = li[0]
            if len(li) != 6:
                print("Length of meeting is not 6. Re-evaluate " + str(li))
            d["TYPE"] = li[1]
            d["DAYS"] = li[2]
            d["TIME"] = li[3]
            d["BUILD"] = li[4]
            d["ROOM"] = li[5]
            
            meeting_list.append(d)
            
        return(meeting_list)
    
    except:
        print("Error when parsing Meeting " + str(big_list))

In [121]:
def parse_enrollment(big_list):
    cols = ['SEAT', 'SID', 'SURNAME', 'PREFNAME', 'LEVEL', 'UNITS', 'CLASS', 'MAJOR', 'GRADE', 'STATUS', 'EMAIL']
    df = pd.DataFrame(columns=cols)
    
    if len(big_list) == 0: # no listings
        return df
    
    #try:
    for li in big_list[1:]:
        if len(li) != 11:
            print("Length of enrollment is not 11. Re-evaluate " + str(li))
        d = {}
        d["SEAT"] = li[0]
        d["SID"] = li[1]
        d["SURNAME"] = li[2]
        d["PREFNAME"] = li[3]
        d["LEVEL"] = li[4]
        d["UNITS"] = li[5]
        d["CLASS"] = li[6]
        d["MAJOR"] = li[7]
        d["GRADE"] = li[8]
        d["STATUS"] = li[9]
        d["EMAIL"] = li[10]
        df = df.append(d, ignore_index=True)
    return(df)
        
    #except:
    #    print("Error when parsing Enrollment " + str(big_list))

In [129]:
tar = tarfile.open("Grades.tgz", "r:gz")
i = 0
courses = {}
for member in tar.getmembers():
    print(member)
    i += 1
    if i > 10: ###### remove this later
        break
        
    f = tar.extractfile(member)
    if f is not None:
        big_list = []
        csv_file = io.StringIO(f.read().decode('ascii'))
        reader = csv.reader(csv_file)
        content = list(reader)
        csv_length = len(content)

        for idx, row in enumerate(content):
            if row != [''] and idx < csv_length:
                big_list.append(row)
                if idx < csv_length - 1:
                    continue
                
            if big_list != []:
                row_type = determine_row_type(big_list)
                if row_type == "CID":
                    info = parse_CID(big_list)
                    (cid, term) = big_list[1][0], big_list[1][1]
                    x = Course(big_list[1][0], big_list[1][1])
                    x.add_course_info(info)
                    
                if row_type == "meet":
                    meeting = parse_meeting(big_list)
                    x.add_meeting(meeting)
                    
                if row_type == "enroll":
                    enroll = parse_enrollment(big_list)
                    x.add_enrollment(enroll)
                    courses[(cid, term)] = x
                    
            if row == ['']:
                big_list = []

<TarInfo 'Grades' at 0x27100d09f20>
<TarInfo 'Grades/2002_Q3.csv' at 0x2716b7ffa70>
<TarInfo 'Grades/2010_Q3.csv' at 0x2716b7ff9a8>
<TarInfo 'Grades/1994_Q4.csv' at 0x2716b7ffd90>
<TarInfo 'Grades/2010_Q2.csv' at 0x2716b7ffe58>
<TarInfo 'Grades/2002_Q2.csv' at 0x2716b7fff20>
<TarInfo 'Grades/1996_Q1.csv' at 0x2716b7ff8e0>
<TarInfo 'Grades/1998_Q4.csv' at 0x2716b8ef110>
<TarInfo 'Grades/1996_Q3.csv' at 0x2716b8ef048>
<TarInfo 'Grades/2010_Q1.csv' at 0x2716b8ef1d8>
<TarInfo 'Grades/2000_Q4.csv' at 0x2716b8ef2a0>


In [ ]:
conn = psycopg2.connect("dbname=fakeu user=postgres")
cur = conn.cursor()

In [ ]:
cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")
cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)", (100, "abc'def"))
cur.execute("SELECT * FROM test;")
cur.fetchone()

In [ ]:
conn.commit()

In [ ]:
cur.close()
conn.close()